# EDS Symptom Discovery - Data Analysis

This notebook visualizes the discovered symptoms and identifies **hidden symptoms** with high patient reports but low research coverage.

## Key Metrics:
- **Surprise Score**: (Patient frequency) × (1 - Research coverage)
- **Evidence Tier**: 1 (Diagnostic) > 2 (Research) > 3 (Patient) > 4 (Emerging)

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
%matplotlib inline

## Load Data

In [ ]:
# Load validated symptom database
with open('../data/validated/llm_symptom_database.json', 'r') as f:
    validated_symptoms = json.load(f)

# Convert to DataFrame
df = pd.DataFrame(validated_symptoms)

# Add helper columns
df['patient_frequency'] = df['reddit_data'].apply(lambda x: x['frequency'])
df['mention_count'] = df['reddit_data'].apply(lambda x: x['mention_count'])
df['paper_count'] = df['pubmed_data'].apply(lambda x: x['paper_count'])

print(f"Loaded {len(df)} validated symptoms")
df.head()

## 1. Top Symptoms by Patient Reports

In [ ]:
# Bar chart of top 15 symptoms
top_symptoms = df.nlargest(15, 'mention_count')[['symptom', 'mention_count', 'patient_frequency']]

fig, ax = plt.subplots(figsize=(12, 8))
bars = ax.barh(top_symptoms['symptom'], top_symptoms['mention_count'], color='steelblue')
ax.set_xlabel('Number of Patient Mentions', fontsize=12)
ax.set_title('Top 15 EDS Symptoms by Patient Reports', fontsize=14, fontweight='bold')
ax.invert_yaxis()

# Add value labels
for i, (bar, freq) in enumerate(zip(bars, top_symptoms['patient_frequency'])):
    ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
            f"{freq*100:.1f}%", va='center', fontsize=10)

plt.tight_layout()
plt.show()

## 2. Surprise Scores - Finding Hidden Symptoms

In [ ]:
# Scatter plot: Patient frequency vs Research coverage
fig, ax = plt.subplots(figsize=(12, 8))

# Color by surprise score
scatter = ax.scatter(df['patient_frequency'], df['paper_count'], 
                     c=df['surprise_score'], s=100, cmap='YlOrRd', 
                     alpha=0.6, edgecolors='black', linewidth=0.5)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Surprise Score', fontsize=12)

# Labels
ax.set_xlabel('Patient Report Frequency', fontsize=12)
ax.set_ylabel('Number of Research Papers', fontsize=12)
ax.set_title('Hidden Symptoms: High Patient Reports + Low Research', 
             fontsize=14, fontweight='bold')

# Annotate high surprise symptoms
high_surprise = df.nlargest(5, 'surprise_score')
for _, row in high_surprise.iterrows():
    ax.annotate(row['symptom'], 
                xy=(row['patient_frequency'], row['paper_count']),
                xytext=(10, 10), textcoords='offset points',
                fontsize=9, bbox=dict(boxstyle='round,pad=0.3', 
                facecolor='yellow', alpha=0.7),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

# Quadrant lines
ax.axhline(y=3, color='gray', linestyle='--', alpha=0.5, label='Research threshold')
ax.axvline(x=0.05, color='gray', linestyle='--', alpha=0.5, label='Frequency threshold')

ax.legend()
plt.tight_layout()
plt.show()

print("\n🎯 Top 5 Hidden Symptoms (High Surprise):")
print(high_surprise[['symptom', 'surprise_score', 'mention_count', 'paper_count']])

## 3. Evidence Tier Distribution

In [ ]:
# Pie chart of evidence tiers
tier_counts = df['evidence_tier'].value_counts().sort_index()
tier_labels = [
    '🏆 Diagnostic Criteria',
    '✅ Research-Backed', 
    '💬 Patient-Validated',
    '⚠️ Emerging Pattern'
]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Pie chart
ax1.pie(tier_counts, labels=[tier_labels[i-1] for i in tier_counts.index], 
        autopct='%1.1f%%', startangle=90, colors=['gold', 'lightgreen', 'lightblue', 'lightcoral'])
ax1.set_title('Evidence Quality Distribution', fontsize=14, fontweight='bold')

# Bar chart
bars = ax2.bar(range(len(tier_counts)), tier_counts.values, 
               color=['gold', 'lightgreen', 'lightblue', 'lightcoral'])
ax2.set_xticks(range(len(tier_counts)))
ax2.set_xticklabels([tier_labels[i-1] for i in tier_counts.index], rotation=45, ha='right')
ax2.set_ylabel('Number of Symptoms', fontsize=12)
ax2.set_title('Symptoms by Evidence Tier', fontsize=14, fontweight='bold')

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height)}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 4. Research Gaps Analysis

In [ ]:
# Identify research gaps: symptoms with >10 patient mentions but <3 papers
research_gaps = df[(df['mention_count'] > 10) & (df['paper_count'] < 3)].copy()
research_gaps = research_gaps.sort_values('mention_count', ascending=False)

if len(research_gaps) > 0:
    print(f"\n🔍 Found {len(research_gaps)} symptoms with research gaps:\n")
    print(research_gaps[['symptom', 'mention_count', 'paper_count', 'surprise_score']].to_string(index=False))
    
    # Visualization
    fig, ax = plt.subplots(figsize=(12, 6))
    x = range(len(research_gaps))
    ax.bar(x, research_gaps['mention_count'], color='coral', label='Patient Mentions')
    ax2 = ax.twinx()
    ax2.plot(x, research_gaps['paper_count'], 'o-', color='steelblue', 
             linewidth=2, markersize=8, label='Research Papers')
    
    ax.set_xlabel('Symptom', fontsize=12)
    ax.set_ylabel('Patient Mentions', fontsize=12, color='coral')
    ax2.set_ylabel('Research Papers', fontsize=12, color='steelblue')
    ax.set_xticks(x)
    ax.set_xticklabels(research_gaps['symptom'], rotation=45, ha='right')
    ax.set_title('Research Gaps: High Patient Reports, Low Research', 
                 fontsize=14, fontweight='bold')
    
    ax.legend(loc='upper left')
    ax2.legend(loc='upper right')
    plt.tight_layout()
    plt.show()
else:
    print("\n✅ No significant research gaps found!")

## 5. Export Results for Frontend

In [ ]:
# Create summary for frontend
summary = {
    'total_symptoms': len(df),
    'high_surprise_count': len(df[df['surprise_score'] >= 0.4]),
    'research_gaps_count': len(research_gaps),
    'top_symptoms': df.nlargest(10, 'mention_count')[['symptom', 'mention_count', 'surprise_score']].to_dict('records'),
    'hidden_gems': df.nlargest(10, 'surprise_score')[['symptom', 'surprise_score', 'mention_count', 'paper_count']].to_dict('records')
}

# Save for frontend
with open('../frontend/data/symptom_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("✅ Summary exported to frontend/data/symptom_summary.json")
print(f"\n📊 Summary Statistics:")
print(f"   Total symptoms validated: {summary['total_symptoms']}")
print(f"   High surprise symptoms: {summary['high_surprise_count']}")
print(f"   Research gaps identified: {summary['research_gaps_count']}")

## 6. Interactive Filtering

In [ ]:
# Filter symptoms by criteria
def filter_symptoms(min_mentions=5, min_surprise=0.0, max_papers=100, tier=None):
    """Filter symptoms by custom criteria."""
    filtered = df[
        (df['mention_count'] >= min_mentions) &
        (df['surprise_score'] >= min_surprise) &
        (df['paper_count'] <= max_papers)
    ]
    
    if tier:
        filtered = filtered[filtered['evidence_tier'] == tier]
    
    return filtered.sort_values('surprise_score', ascending=False)

# Example: Find symptoms with high surprise and low research
print("🎯 Symptoms with high surprise (>0.3) and few papers (<5):\n")
hidden = filter_symptoms(min_mentions=5, min_surprise=0.3, max_papers=5)
print(hidden[['symptom', 'mention_count', 'paper_count', 'surprise_score', 'tier_label']])